# Deploying models as SQL functions

In [0]:
import tensorflow
import keras
import h5py

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
Using TensorFlow backend.

## Download the flower dataset

In [0]:
%sh
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz &>/dev/null
cd flower_photos
ls
pwd

In [0]:
%sh
rm -rf /databricks/driver/flower_photos/sample_images
cd flower_photos
mkdir sample_images
cp /databricks/driver/flower_photos/daisy/100080576_f52e8ee070_n.jpg sample_images/
cp /databricks/driver/flower_photos/daisy/10140303196_b88d3d6cec.jpg sample_images/
cp /databricks/driver/flower_photos/daisy/10172379554_b296050f82_n.jpg sample_images/

In [0]:
%sh
ls -l /databricks/driver/flower_photos/sample_images

total 180
-rw-r----- 1 root root 26797 Jan 14 11:23 100080576_f52e8ee070_n.jpg
-rw-r----- 1 root root 117247 Jan 14 11:23 10140303196_b88d3d6cec.jpg
-rw-r----- 1 root root 36410 Jan 14 11:23 10172379554_b296050f82_n.jpg

## Load images into Dataframe

In [0]:
sample_dir = 'file:/databricks/driver/flower_photos/sample_images'

In [0]:
df = spark.read.format('image').load(sample_dir)
df.show()
df.printSchema()

+--------------------+
 image|
+--------------------+
[file:/databricks...|
[file:/databricks...|
[file:/databricks...|
+--------------------+

root
-- image: struct (nullable = true)
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)
 |-- nChannels: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- data: binary (nullable = true)

## Registering SQL User Defined Function

In [0]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("inceptionV3_udf_with_preprocessing", InceptionV3(weights="imagenet"), keras_load_img)

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4271: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5

 8192/96112376 [..............................] - ETA: 1s
 24576/96112376 [..............................] - ETA: 4:03
 57344/96112376 [..............................] - ETA: 3:29
 106496/96112376 [..............................] - ETA: 2:49
 229376/96112376 [..............................] - ETA: 1:44
 442368/96112376 [..............................] - ETA: 1:07
 909312/96112376 [..............................] - ETA: 39s 
 1826816/96112376 [..............................] - ETA: 22s
 3186688/96112376 [..............................] - ETA: 14s
 4661248/96112376 [>.............................] - ETA: 11s
 6053888/96112376 [>.............................] - ETA: 9s 
 7610368/96112376 [=>............................] - ETA: 8s
 8871936/96112376 [=>............................] - ETA: 7s
10461184/96112376 [==>...........................] - ETA: 6s
11640832/96112376 [==>...........................] - ETA: 6s
13230080/96112376 [===>..........................] - ETA: 5s
14458880/96112376 [===>..........................] - ETA: 5s
16031744/96112376 [====>.........................] - ETA: 5s
17440768/96112376 [====>.........................] - ETA: 5s
19013632/96112376 [====>......

In [0]:
image_df = spark.read.format('image').load(sample_dir)
image_df.createOrReplaceTempView('vw_sample_images')
image_df.show()

+--------------------+
 image|
+--------------------+
[file:/databricks...|
[file:/databricks...|
[file:/databricks...|
+--------------------+

In [0]:
%sql 

SELECT inceptionV3_udf_with_preprocessing(image) as predictions from vw_sample_images

predictions


##### Decoding the output

In [0]:
import numpy as np
from keras.applications.inception_v3 import preprocess_input, decode_predictions

In [0]:
out = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.99999857, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.4782206e-35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.126583e-35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.823431e-35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.1818338e-37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.4891264e-27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0000013868001, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.0348651e-36, 0, 0, 0, 0, 0, 0, 0, 1.7536236e-26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.802739e-32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
out  = np.asarray(out)
print('Predicted:', decode_predictions(out, top=3)[0])

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1598199382691499> in <module> 
 1 out = np . asarray ( out ) 
 ----> 2 print ( 'Predicted:' , decode_predictions ( out , top = 3 ) [ 0 ] ) 

 /databricks/python/lib/python3.7/site-packages/keras/applications/__init__.py in wrapper (*args, **kwargs) 
 18 kwargs [ 'models' ] = models
 19 kwargs [ 'utils' ] = utils
 ---> 20 return base_fun ( * args , ** kwargs ) 
 21 
 22 return wrapper

 /databricks/python/lib/python3.7/site-packages/keras/applications/inception_v3.py in decode_predictions (*args, **kwargs) 
 14 @ keras_modules_injection
 15 def decode_predictions ( * args , ** kwargs ) : 
 ---> 16 return inception_v3 . decode_predictions ( * args , ** kwargs ) 
 17 
 18 

 /databricks/python/lib/python3.7/site-packages/keras_applications/imagenet_utils.py in decode_predictions (preds, top, **kwargs) 
 220 'a batch of predictions ' 
 221 '(i.e. a 2D array of shape (samples, 1000)). ' 
 --> 222 'Found array with shape: ' + str(preds.shape))
 223 if CLASS_INDEX is None : 
 224 fpath = keras_utils.get_file(

 ValueError : `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1000,)

In [0]:
out = out.reshape(-1,1000)
out.shape

Out[14]: (1, 1000)

In [0]:
print('Predicted: ', decode_predictions(out, top=3)[0])

Predicted: [('n01924916', 'flatworm', 0.99999857), ('n03950228', 'pitcher', 1.3868001e-06), ('n04131690', 'saltshaker', 1.7536236e-26)]